In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers,optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
def build_model():
    model = models.Sequential()

 # 1st block
    model.add(layers.Conv2D(32, (3,3), padding='same', input_shape=(32,32,3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same',activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.2))
 # 2nd block
    model.add(layers.Conv2D(64, (3,3), padding='same',activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same',activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3))
 # 3d block
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.4))
    # dense
    model.add(layers.Flatten())
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
    model.summary()
    return model

In [3]:
EPOCHS=1
NUM_CLASSES = 10
def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
     # normalize
    mean = np.mean(x_train,axis=(0,1,2,3))
    std = np.std(x_train,axis=(0,1,2,3))
    x_train = (x_train-mean)/(std+1e-7)
    x_test = (x_test-mean)/(std+1e-7)
    y_train = tf.keras.utils.to_categorical(y_train,NUM_CLASSES)
    y_test = tf.keras.utils.to_categorical(y_test,NUM_CLASSES)
    return x_train, y_train, x_test, y_test

In [7]:
(x_train, y_train, x_test, y_test) = load_data()
model = build_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        1

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
# train
batch_size = 64
model.fit(x_train, y_train, batch_size=batch_size, epochs=EPOCHS, validation_data=(x_test,y_test))
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

In [9]:
datagen = ImageDataGenerator(rotation_range = 30, width_shift_range = 0.2, height_shift_range = 0.2,
                            horizontal_flip = True, zoom_range = 0.2)
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

In [10]:
model.fit(datagen.flow(x_train, y_train, batch_size = batch_size), epochs = EPOCHS, verbose = 1,
         validation_data = (x_test, y_test))

In [23]:
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model_json = model.to_json()
with open('model.json','w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5')

157/157 [==============================] - 2s 15ms/step - loss: 2.3027 - accuracy: 0.1172

Test score: 2.3027310371398926
Test accuracy: 0.11720000207424164


In [11]:
# import numpy as np
# import scipy.misc
# from tensorflow.keras.models import model_from_json
# from tensorflow.keras.optimizers import SGD
# # load model
# model_architecture = 'cifar10_architecture.json'
# model_weights = 'cifar10_weights.h5'
# model = model_from_json(open(model_architecture).read())
# model.load_weights(model_weights)

In [13]:
from tensorflow.keras.applications import VGG19,ResNet50

In [15]:
base_model_1 = VGG19(include_top=False,weights='imagenet',input_shape=(32,32,3), classes=10,)
base_model_1.trainable = False

In [16]:
model_1 = models.Sequential()
model_1.add(base_model_1)
model_1.add(layers.Flatten())

model_1.add(layers.Dense(1024, activation = 'relu', input_dim = 512))
model_1.add(layers.Dense(512,activation=('relu'))) 
model_1.add(layers.Dense(256,activation=('relu'))) 
model_1.add(layers.Dense(128,activation=('relu')))
model_1.add(layers.Dense(10,activation=('softmax'))) #This is the classification layer

In [17]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

In [19]:
model_1.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model_1.fit(datagen.flow(x_train, y_train, batch_size = 64), epochs = 10, verbose = 1,
           validation_data = (x_test, y_test))

Epoch 1/10
782/782 [==============================] - 115s 146ms/step - loss: 1.4626 - accuracy: 0.4835 - val_loss: 1.3085 - val_accuracy: 0.5415
Epoch 2/10
782/782 [==============================] - 114s 145ms/step - loss: 1.3116 - accuracy: 0.5428 - val_loss: 1.2085 - val_accuracy: 0.5827
Epoch 3/10
782/782 [==============================] - 114s 145ms/step - loss: 1.2667 - accuracy: 0.5592 - val_loss: 1.2337 - val_accuracy: 0.5840
Epoch 4/10
782/782 [==============================] - 118s 150ms/step - loss: 1.2486 - accuracy: 0.5663 - val_loss: 1.1416 - val_accuracy: 0.6108
Epoch 5/10
782/782 [==============================] - 123s 158ms/step - loss: 1.2304 - accuracy: 0.5743 - val_loss: 1.1242 - val_accuracy: 0.6201
Epoch 6/10
782/782 [==============================] - 117s 149ms/step - loss: 1.2109 - accuracy: 0.5810 - val_loss: 1.1244 - val_accuracy: 0.6080
Epoch 7/10
782/782 [==============================] - 115s 147ms/step - loss: 1.2079 - accuracy: 0.5836 - val_loss: 1.1068 -

In [51]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

imgs = ['cat.jpg', 'dog.jpg']

imgs = [tf.keras.preprocessing.image.load_img(img, target_size=(32,32)) for img in imgs]
imgs = [tf.keras.preprocessing.image.img_to_array(img) for img in imgs]
imgs = [np.expand_dims(img, axis = 0) for img in imgs]
imgs = np.array(imgs).reshape(-1,32,32,3)

predictions = model_1.predict_classes(imgs)
predictions

array([4, 4], dtype=int64)

# Very deep convolutional networks for large-scale image recognition

In [65]:
def VGG_16(weights_path = None):
    model = models.Sequential()
    model.add(layers.ZeroPadding2D((1,1), input_shape = (224, 224, 3)))
    model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
    model.add(layers.MaxPooling2D((2,2), strides = (2,2)))
    
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
    model.add(layers.MaxPooling2D((2,2), strides = (2,2)))
    
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(256, (3, 3), activation = 'relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(256, (3, 3), activation = 'relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(256, (3, 3), activation = 'relu'))
    model.add(layers.MaxPooling2D((2,2), strides = (2,2)))
    
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(512, (3, 3), activation = 'relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(512, (3, 3), activation = 'relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(512, (3, 3), activation = 'relu'))
    model.add(layers.MaxPooling2D((2,2), strides = (2,2)))
    
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(512, (3, 3), activation = 'relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(512, (3, 3), activation = 'relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(512, (3, 3), activation = 'relu'))
    model.add(layers.MaxPooling2D((2,2), strides = (2,2)))
    
    model.add(layers.Flatten())
    #top layer of the VGG net
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1000, activation='softmax'))
    
    if weights_path:
        model.load_weights(weights_path)
    return model

In [66]:
import cv2
im = cv2.resize(cv2.imread('cat.jpg'), (224, 224))
#im = im.transpose((2,0,1))
im = np.expand_dims(im, axis=0)
# Test pretrained model
model = VGG_16('vgg16_weights_tf_dim_ordering_tf_kernels.h5')
model.summary()
model.compile(optimizer='sgd', loss='categorical_crossentropy')
out = model.predict(im)
print(np.argmax(out))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_14 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 112, 112, 128)    